## 네이버 뉴스 개편 되기 전

In [37]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_news(sectionid, date):
    url = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId=" + str(sectionid) + "&date=" + str(date)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    l = []
    ranking_text = soup.find_all(class_ = 'ranking_text')
    for item in ranking_text:
        d = {}
        d['LinkSrc'] = item.find('a')['href']
        d['Title'] = item.find('a')['title']
        d['Views'] = item.find(class_ = "ranking_view").get_text()
        l.append(d)
    
    for link in l:
        resp = requests.get("http://news.naver.com" + link['LinkSrc'])
        soup = BeautifulSoup(resp.text, "html.parser")
        content = soup.find(id="articleBodyContents")
        link['Content'] = clean_text(content)
        
    df = pd.DataFrame(l)
    return df
    
# text 정제하기
def clean_text(text):
    content = text.get_text()
    cleaned_text = re.sub('[a-zA-Z]', '', content)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    cleaned_text = cleaned_text.replace("오류를 우회하기 위한 함수 추가 ", "")
    cleaned_text = cleaned_text.replace("동영상 뉴스 오류를 우회하기 위한 함수 추가 ", "")
    cleaned_text = cleaned_text.replace("무단전재 및 재배포 금지", "")
    return cleaned_text

## 네이버 뉴스 개편된 이후

In [16]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# error 발생
url = "https://news.naver.com/main/ranking/office.nhn?officeId=214" 
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.pareser")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### 언론사 ID

In [30]:
press_ID = {"MBC" : "214", "KBS" : "056", "SBS" : "055"}

### 텍스트 처리

In [2]:
def clean_text(text):
    content = text.get_text()
    cleaned_text = re.sub('[a-zA-Z]', '', content)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    cleaned_text = cleaned_text.replace("오류를 우회하기 위한 함수 추가 ", "")
    cleaned_text = cleaned_text.replace("무단전재 및 재배포 금지", "")
    cleaned_text = cleaned_text.replace("동영상 뉴스                       뉴스데스크 ", "")
    cleaned_text = cleaned_text.replace("동영상 뉴스                       뉴스투데이 ", "")
    cleaned_text = cleaned_text.replace("앵커  ", "")
    return cleaned_text

### 뉴스 크롤링

In [5]:
# demo v0.1
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

url = "https://news.naver.com/main/ranking/office.nhn?officeId=214&date=20210114"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

ranking_box = soup.find_all(class_="rankingnews_box_inner")
l = []

rank = ranking_text[0].find_all(class_ = "list_ranking_num")
url_list = ranking_text[0].find_all(class_="list_content")

for num in range(20):
    d = {}
    d['Rank'] = rank[num].get_text()
    d['URL'] = url_list[num].find('a')['href']
    d['Title'] = url_list[num].find('a').get_text()
    d['View'] = url_list[num].find(class_ = "list_view").get_text()
    l.append(d)

for link in l:
    resp = requests.get("http://news.naver.com" + link['URL'], headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    content = soup.find(id="articleBodyContents")
    link['Content'] = clean_text(content)

df = pd.DataFrame(l)
df.to_csv("20210112 MBC 많이 본 뉴스.csv", sep=",", index=False, encoding="utf-8-sig")




rank = ranking_text[1].find_all(class_ = "list_ranking_num")
url_list = ranking_text[1].find_all(class_="list_content")

for num in range(20):
    d = {}
    d['Rank'] = rank[num].get_text()
    d['URL'] = url_list[num].find('a')['href']
    d['Title'] = url_list[num].find('a').get_text()
    d['Comment'] = url_list[num].find(class_ = "list_comment nclicks('RBP.dcmtnwscmt')").get_text()
    l.append(d)

for link in l:
    resp = requests.get("http://news.naver.com" + link['URL'], headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    content = soup.find(id="articleBodyContents")
    link['Content'] = clean_text(content)

df = pd.DataFrame(l)
df.to_csv("20210112 MBC 많이 본 뉴스.csv", sep=",", index=False, encoding="utf-8-sig")

In [43]:
# demo v0.2 (2021.01.14, MBC)
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

url = "https://news.naver.com/main/ranking/office.nhn?officeId=214&date=20210114"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

ranking_box = soup.find_all(class_="rankingnews_box_inner")
l = []

for ranking_type in range(2):
    ranking = ranking_box[ranking_type].find_all(class_="list_ranking_num")
    url_list = ranking_box[ranking_type].find_all(class_="list_content")

    for rank in range(20):
        d = {}
        d['Rank'] = ranking[rank].get_text()
        d['URL'] = url_list[rank].find('a')['href']
        d['Title'] = url_list[rank].find('a').get_text()
        if (ranking_type == 0):
            d['View'] = url_list[rank].find(class_="list_view").get_text()
        elif (ranking_type == 1):
            d['Comment'] = url_list[rank].find(class_="list_comment nclicks('RBP.dcmtnwscmt')").get_text()
        l.append(d)

for news in l:
    resp = requests.get("https://news.naver.com" + news['URL'], headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    content = soup.find(id="articleBodyContents")
    news['Content'] = clean_text(content)

df = pd.DataFrame(l)
df.to_csv("20210114_MBC_ranking_news.csv", sep=",", index=False, encoding="utf-8-sig")

In [5]:
# demo v0.3 (2021.01.15, MBC, KBS, SBS)
import os
os.chdir(r"C:/Users/cjy89/NLP/Naver_news_crawling/")
press_ID = {"MBC" : "214", "KBS" : "056", "SBS" : "055"}

import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup


def clean_text(text):
    content = text.get_text()
    cleaned_text = re.sub('[a-zA-Z]', '', content)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    cleaned_text = cleaned_text.replace("오류를 우회하기 위한 함수 추가 ", "")
    cleaned_text = cleaned_text.replace("무단전재 및 재배포 금지", "")
    cleaned_text = cleaned_text.replace("동영상 뉴스                       뉴스데스크 ", "")
    cleaned_text = cleaned_text.replace("동영상 뉴스                       뉴스투데이 ", "")
    cleaned_text = cleaned_text.replace("앵커  ", "")
    return cleaned_text

for press in press_ID:
    start = time.time()
    url = "https://news.naver.com/main/ranking/office.nhn?officeId=" + press_ID[press] + "&date=20210115"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    ranking_box = soup.find_all(class_="rankingnews_box_inner")
    l = []

    for ranking_type in range(2):
        ranking = ranking_box[ranking_type].find_all(class_="list_ranking_num")
        url_list = ranking_box[ranking_type].find_all(class_="list_content")

        for rank in range(20):
            d = {}
            d['Rank'] = ranking[rank].get_text()
            d['URL'] = url_list[rank].find('a')['href']
            d['Title'] = url_list[rank].find('a').get_text()
            if (ranking_type == 0):
                d['View'] = url_list[rank].find(class_="list_view").get_text()
            elif (ranking_type == 1):
                d['Comment'] = url_list[rank].find(class_="list_comment nclicks('RBP.dcmtnwscmt')").get_text()
            l.append(d)

    for news in l:
        resp = requests.get("https://news.naver.com" + news['URL'], headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        content = soup.find(id="articleBodyContents")
        news['Content'] = clean_text(content)

    df = pd.DataFrame(l)
    title = press + "/20210115_" + press + "_ranking_news.csv"
    df.to_csv(title, sep=",", index=False, encoding="utf-8-sig")
    end = time.time()
    print("Crawling " + press + " news :", end - start)

Crawling MBC news : 10.095808267593384
Crawling KBS news : 8.139859437942505
Crawling SBS news : 8.135042905807495


In [31]:
# demo v0.4 날짜를 입력받아 자동으로 크롤링
import os
os.chdir(r"C:\Users\cjy89\NLP\Project_news_crawling\Naver")

from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

press_ID = {"MBC": "214", "KBS": "056", "SBS": "055", "JTBC": "437"}

"""
def clean_text(text):
    content = text.get_text()
    cleaned_text = re.sub('[a-zA-Z]', '', content)
    cleaned_text = re.sub(
        '[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace(
        "🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    cleaned_text = cleaned_text.replace("오류를 우회하기 위한 함수 추가 ", "")
    cleaned_text = cleaned_text.replace("무단전재 및 재배포 금지", "")
    cleaned_text = cleaned_text.replace(
        "동영상 뉴스                       뉴스데스크 ", "")
    cleaned_text = cleaned_text.replace(
        "동영상 뉴스                       뉴스투데이 ", "")
    cleaned_text = cleaned_text.replace("앵커  ", "")
    return cleaned_text
"""

# 8자리로 된 날짜를 입력하면 해당 날짜의 ranking news를 가져온다.
def get_ranking_news(date):
    total_time = 0
    
    # 언론사 순
    for press in press_ID:
        start = time.time()
        url = "https://news.naver.com/main/ranking/office.nhn?officeId=" + press_ID[press] + "&date=" + str(date)
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")

        ranking_box = soup.find_all(class_="rankingnews_box_inner")
        l = []

        # 조회수 -> 댓글 수
        for ranking_type in range(2):
            ranking = ranking_box[ranking_type].find_all(class_="list_ranking_num")
            url_list = ranking_box[ranking_type].find_all(class_="list_content")

            # 랭킹 순 (1 ~ 20)
            for rank in range(20):
                d = {}
                d['Date'] = int(date)
                d['Press'] = press
                d['Rank'] = ranking[rank].get_text()
                d['URL'] = url_list[rank].find('a')['href']
                d['Title'] = url_list[rank].find('a').get_text()
                if (ranking_type == 0):
                    d['View'] = url_list[rank].find(class_="list_view").get_text()
                elif (ranking_type == 1):
                    d['Comment'] = url_list[rank].find(class_="list_comment nclicks('RBP.dcmtnwscmt')").get_text()
                l.append(d)

        # 본문 가져오기
        for news in l:
            resp = requests.get("https://news.naver.com" + news['URL'], headers=headers)
            soup = BeautifulSoup(resp.text, "html.parser")
            news['Content'] = soup.find(id="articleBodyContents").get_text()
            # news['Content'] = clean_text(content)

        df = pd.DataFrame(l)
        title = press + "/" + str(date) + "_" + press + "_ranking_news.csv"
        df.to_csv(title, sep=",", index=False, encoding="utf-8-sig")
        end = time.time()
        total_time += end - start
        print("Crawling " + str(date) + " " + press + " news :", end - start)
    print("Total time :", total_time)
    print("Average time : ", total_time/4)
    print("───────────────────")

In [32]:
for i in range(24):
    get_ranking_news(20210101+i)

Crawling 20210101 MBC news : 8.836026191711426
Crawling 20210101 KBS news : 8.432084798812866
Crawling 20210101 SBS news : 29.567252159118652
Crawling 20210101 JTBC news : 8.562555313110352
Total time : 55.397918462753296
Average time :  13.849479615688324
───────────────────
Crawling 20210102 MBC news : 8.626611709594727
Crawling 20210102 KBS news : 8.309682130813599
Crawling 20210102 SBS news : 8.432120561599731
Crawling 20210102 JTBC news : 8.853129625320435
Total time : 34.22154402732849
Average time :  8.555386006832123
───────────────────
Crawling 20210103 MBC news : 8.963973760604858
Crawling 20210103 KBS news : 8.656991243362427
Crawling 20210103 SBS news : 8.908329725265503
Crawling 20210103 JTBC news : 8.577554941177368
Total time : 35.106849670410156
Average time :  8.776712417602539
───────────────────
Crawling 20210104 MBC news : 8.909151077270508
Crawling 20210104 KBS news : 9.606096029281616
Crawling 20210104 SBS news : 8.688083410263062
Crawling 20210104 JTBC news : 8.9

## 자연어처리

In [14]:
# MBC 데이터 불러오기 -> 조회수 별, 댓글 수 별 따로 저장
import os
import pandas as pd
os.chdir(r"C:\Users\cjy89\NLP\Project_news_crawling\Naver")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

crawling_list = os.listdir('MBC/')
MBC_ranking_view = pd.DataFrame()
MBC_ranking_comment = pd.DataFrame()

for item in crawling_list:
    df = pd.read_csv("MBC/"+item, sep=",", encoding="utf-8-sig", engine="python")
    MBC_ranking_view = pd.concat([MBC_ranking_view, df.iloc[:20, :]], axis = 0, ignore_index=True)
    MBC_ranking_comment = pd.concat([MBC_ranking_comment, df.iloc[20:, :]], axis=0, ignore_index=True)
    
MBC_ranking_view.drop(["Comment"], axis=1, inplace = True)
MBC_ranking_comment.drop(["View"], axis=1, inplace = True)

In [15]:
MBC_ranking_view

,Date,Rank,URL,Title,View,Content
0,20210101,1,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,신규 확진 다시 1천 명대…내일 거리두기 단계 조정,66220.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[정오뉴스]◀ 앵커 ▶새해 첫...
1,20210101,2,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,모더나 백신 2천만 명분 계약…2분기부터 접종,57823.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶코로나...
2,20210101,3,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,심상찮은 '의사 동호회 감염'…병원 6곳 전파,51620.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶올해 ...
3,20210101,4,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,[MBC여론조사①] 국정운영 '부정 평가' 52.5% vs '긍정 평가' 43.4%,40400.0,\n\n\n\n\n자료사진문재인 대통령의 국정운영에 대해 부정적인 평가가 과반인 5...
4,20210101,5,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,새해 해맞이 명소 폐쇄…이 시각 울산 간절곶,40278.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶202...
5,20210101,6,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,'흰 소띠의 해' 알린 대한민국 1호들,39561.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶코로나...
6,20210101,7,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,1천 명 넘을 듯…내일 거리두기 단계 결정,34867.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶코로나...
7,20210101,8,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,'군기' 잡으려 새벽에 소집…위치 추적까지?,33052.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶배달 ...
8,20210101,9,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"모더나 ""5월부터 한국에 코로나19 백신 공급""",28675.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[정오뉴스]제약사 모더나가 오...
9,20210101,10,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,조마조마한 '1천 명대'…내일 거리두기 재연장?,28554.0,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶코로나...


In [16]:
MBC_ranking_comment

,Date,Rank,URL,Title,Content,Comment
0,20210101,1,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"""文 대통령 속 간첩사상""…전광훈 '막말' 재시동",\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶어제 ...,803.0
1,20210101,2,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,[MBC여론조사①] 국정운영 '부정 평가' 52.5% vs '긍정 평가' 43.4%,\n\n\n\n\n자료사진문재인 대통령의 국정운영에 대해 부정적인 평가가 과반인 5...,707.0
2,20210101,3,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,차기 대권 '3강'…이재명·윤석열·이낙연 초박빙,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶여러분...,459.0
3,20210101,4,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"문 대통령 ""국민 일상의 회복으로 보답하겠다""","\n\n\n\n\n문재인 대통령은 2021년 신축년 새해엔 ""모두의 삶이 코로나로부...",319.0
4,20210101,5,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,모더나 백신 2천만 명분 계약…2분기부터 접종,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶코로나...,314.0
5,20210101,6,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"'의사 시험' 결국 허용…""의료 공백 막아야""",\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶의대 ...,308.0
6,20210101,7,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,신규 확진 다시 1천 명대…내일 거리두기 단계 조정,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[정오뉴스]◀ 앵커 ▶새해 첫...,281.0
7,20210101,8,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"모더나도 계약 완료…""9월까지 1차 접종 마무리""",\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스데스크]◀ 앵커 ▶예고한...,236.0
8,20210101,9,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,"모더나 ""5월부터 한국에 코로나19 백신 공급""",\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[정오뉴스]제약사 모더나가 오...,208.0
9,20210101,10,/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=0...,[MBC여론조사②] 백신준비 '잘하고 있다' 48.7%…'잘못하고 있다' 48.8%,\n\n\n\n\n자료사진정부의 코로나19 백신 접종 준비에 대해 긍정평가와 부정평...,195.0


In [29]:
MBC_ranking_view.loc[:20, 'Content'][1]

'\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[뉴스투데이]◀ 앵커 ▶코로나19가 전 세계를 뒤덮은 2020년, 마지막 날에 정부는 미국 모더나 사와 백신 계약을 완료했습니다.물량은 앞서 발표했던 대로 2천만명분이고, 미리 확보한 백신까지 합치면 인구 수를 넘는 5천 6백만명이 맞을 수 있는 분량이 됐습니다.박영일 기자입니다.◀ 리포트 ▶정부가 모더나 사에서 구매한 코로나19 백신은 4천만번 맞을 수 있는 양입니다.한 사람이 두 번씩 맞아야 하기 때문에 2천만명분에 해당합니다.문재인 대통령과 스테판 반셀 모더나 최고경영자가 지난달 28일 영상 통화로 합의한 뒤, 후속 협상으로 계약을 맺었습니다.지난해 10월 백신 공동 구매 협의체인 \'코백스 퍼실리티\'에서 1천만명분.11월에는 아스트라제네카 백신 1천만명분.지난달 화이자 백신 1천만명분과 얀센 백신 6백만명분까지.[정은경/질병관리청장 (어제)]"모더나 백신까지 우리 국민 총 5천 6백만 명이 접종을 받을 수 있는 물량에 대한 백신 계약을 완료하였습니다."모더나 백신 분량은 당초 추진하던 양보다 두 배로 늘었고, 공급 시기는 올해 3분기에서 2분기로 앞당겼습니다.정은경 질병관리청장은 인구보다 많은 백신을 확보했기 때문에 "집단 면역을 확보하는 데는 충분한 물량"이라고 설명했습니다.특히 최근 확진자가 크게 감소하지 않고 있어, 정부는 백신 공급 시기를 앞당기기 위해 지속적으로 노력하겠다고 밝혔습니다.[정은경/질병관리청장 (어제)]"아스트라제네카는 2021년 1분기부터, 얀센과 모더나는 2분기부터, 화이자는 3분기로 단계적으로 물량이 도입될 예정입니다."정부는 무엇보다 안전하게 백신을 접종하는 일이 중요하다고 보고, 우선 접종 대상자 약 3천만명을 포함해 이번 달 안에 접종 계획을 발표할 예정입니다.MBC뉴스 박영일입니다.MBC뉴스는 24시간 여러분의 제보를 기다립니다.▷ 전화 02-784-4000▷ 이메일 mbcjebo@mbc.co.kr▷ 카카오톡 @mbc제보박영일 기자(parkyi75@imbc.com